# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments  \
0  Jul 19 2022  8:30AM  242155        10   8291639   
1  Jul 19 2022  8:30AM  242145        10   8292170   
2  Jul 19 2022  8:30AM  242144        10   8292175   
3  Jul 19 2022  8:30AM  242149        10   8292178   
4  Jul 18 2022  3:57PM  242154        21  620261-1   
5  Jul 18 2022  3:52PM  242146        16   8290869   
6  Jul 18 2022  3:52PM  242146        16   8291446   
7  Jul 18 2022  3:37PM  242147        21    621119   
8  Jul 18 2022  3:26PM  242142        12    HH6430   
9  Jul 18 2022  3:26PM  242142        12    HH6432   

                         Customer ShipmentStatus  
0                  ClearSpec, LLC       Released  
1               Yusen – 3D Matrix       Released  
2               Yusen – 3D Matrix       Released  
3               Yusen – 3D Matrix       Released  
4               NBTY Global, Inc.       Released  
5  Sartorius Bioprocess Solutions       Released  
6  Sartorius Bioprocess Solutions       Released  
7               NBTY Global, Inc.       Released  
8                       Hush Hush      Executing  
9                       Hush Hush      Executing

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
50  242146       Released          2
51  242147       Released          1
52  242149       Released          1
53  242154       Released          1
54  242155       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
242146                NaN        NaN       2.0
242147                NaN        NaN       1.0
242149                NaN        NaN       1.0
242154                NaN        NaN       1.0
242155                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
241900                0.0        6.0       6.0
241972                1.0        9.0       4.0
242005                0.0        0.0       3.0
242007                0.0        0.0       1.0
242008                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
241900                  0          6         6
241972                  1          9         4
242005                  0          0         3
242007                  0          0         1
242008                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               241900          0          6         6
1               241972          1          9         4
2               242005          0          0         3
3               242007          0          0         1
4               242008          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               241900                   6        6
1               241972         1         9        4
2               242005                            3
3               242007                            1
4               242008                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Jul 19 2022  8:30AM  242155        10                  ClearSpec, LLC
1   Jul 19 2022  8:30AM  242145        10               Yusen – 3D Matrix
2   Jul 19 2022  8:30AM  242144        10               Yusen – 3D Matrix
3   Jul 19 2022  8:30AM  242149        10               Yusen – 3D Matrix
4   Jul 18 2022  3:57PM  242154        21               NBTY Global, Inc.
5   Jul 18 2022  3:52PM  242146        16  Sartorius Bioprocess Solutions
7   Jul 18 2022  3:37PM  242147        21               NBTY Global, Inc.
8   Jul 18 2022  3:26PM  242142        12                       Hush Hush
10  Jul 18 2022  3:22PM  242139        20        ACI Healthcare USA, Inc.
78  Jul 18 2022  3:21PM  242140        10               Yusen – 3D Matrix

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Jul 19 2022  8:30AM  242155        10                  ClearSpec, LLC   
1  Jul 19 2022  8:30AM  242145        10               Yusen – 3D Matrix   
2  Jul 19 2022  8:30AM  242144        10               Yusen – 3D Matrix   
3  Jul 19 2022  8:30AM  242149        10               Yusen – 3D Matrix   
4  Jul 18 2022  3:57PM  242154        21               NBTY Global, Inc.   
5  Jul 18 2022  3:52PM  242146        16  Sartorius Bioprocess Solutions   
6  Jul 18 2022  3:37PM  242147        21               NBTY Global, Inc.   
7  Jul 18 2022  3:26PM  242142        12                       Hush Hush   
8  Jul 18 2022  3:22PM  242139        20        ACI Healthcare USA, Inc.   
9  Jul 18 2022  3:21PM  242140        10               Yusen – 3D Matrix   

  Completed Executing Released  
0                            1  
1                            1  
2                            1  
3                            1  
4                            1  
5                            2  
6                            1  
7                   2           
8                           68  
9                   1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Jul 19 2022  8:30AM  242155        10                  ClearSpec, LLC   
1  Jul 19 2022  8:30AM  242145        10               Yusen – 3D Matrix   
2  Jul 19 2022  8:30AM  242144        10               Yusen – 3D Matrix   
3  Jul 19 2022  8:30AM  242149        10               Yusen – 3D Matrix   
4  Jul 18 2022  3:57PM  242154        21               NBTY Global, Inc.   
5  Jul 18 2022  3:52PM  242146        16  Sartorius Bioprocess Solutions   
6  Jul 18 2022  3:37PM  242147        21               NBTY Global, Inc.   
7  Jul 18 2022  3:26PM  242142        12                       Hush Hush   
8  Jul 18 2022  3:22PM  242139        20        ACI Healthcare USA, Inc.   
9  Jul 18 2022  3:21PM  242140        10               Yusen – 3D Matrix   

  Released Executing Completed  
0        1                      
1        1                      
2        1                      
3        1                      
4        1                      
5        2                      
6        1                      
7                  2            
8       68                      
9                  1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse           Customer Released  \
0  Jul 19 2022  8:30AM  242155        10     ClearSpec, LLC        1   
1  Jul 19 2022  8:30AM  242145        10  Yusen – 3D Matrix        1   
2  Jul 19 2022  8:30AM  242144        10  Yusen – 3D Matrix        1   
3  Jul 19 2022  8:30AM  242149        10  Yusen – 3D Matrix        1   
4  Jul 18 2022  3:57PM  242154        21  NBTY Global, Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Jul 19 2022  8:30AM  242155        10     ClearSpec, LLC       1.0   
1  Jul 19 2022  8:30AM  242145        10  Yusen – 3D Matrix       1.0   
2  Jul 19 2022  8:30AM  242144        10  Yusen – 3D Matrix       1.0   
3  Jul 19 2022  8:30AM  242149        10  Yusen – 3D Matrix       1.0   
4  Jul 18 2022  3:57PM  242154        21  NBTY Global, Inc.       1.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Jul 19 2022  8:30AM  242155        10     ClearSpec, LLC       1.0   
1  Jul 19 2022  8:30AM  242145        10  Yusen – 3D Matrix       1.0   
2  Jul 19 2022  8:30AM  242144        10  Yusen – 3D Matrix       1.0   
3  Jul 19 2022  8:30AM  242149        10  Yusen – 3D Matrix       1.0   
4  Jul 18 2022  3:57PM  242154        21  NBTY Global, Inc.       1.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3631341      38.0       33.0       12.0
102         242008       1.0        0.0        0.0
12          484277       0.0        3.0        0.0
16          242146       2.0        0.0        0.0
19          968514      11.0       15.0        0.0
20         1694282     142.0       24.0        2.0
21         3631662      15.0        0.0        0.0
22          484232       2.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3631341      38.0       33.0       12.0
1       102   242008       1.0        0.0        0.0
2        12   484277       0.0        3.0        0.0
3        16   242146       2.0        0.0        0.0
4        19   968514      11.0       15.0        0.0
5        20  1694282     142.0       24.0        2.0
6        21  3631662      15.0        0.0        0.0
7        22   484232       2.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      38.0       33.0       12.0
1       102       1.0        0.0        0.0
2        12       0.0        3.0        0.0
3        16       2.0        0.0        0.0
4        19      11.0       15.0        0.0
5        20     142.0       24.0        2.0
6        21      15.0        0.0        0.0
7        22       2.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   38.0
1       102  Released    1.0
2        12  Released    0.0
3        16  Released    2.0
4        19  Released   11.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10  102   12   16    19     20    21   22
Status                                                
Completed  12.0  0.0  0.0  0.0   0.0    2.0   0.0  0.0
Executing  33.0  0.0  3.0  0.0  15.0   24.0   0.0  0.0
Released   38.0  1.0  0.0  2.0  11.0  142.0  15.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10  102   12   16    19     20    21   22
0          Completed  12.0  0.0  0.0  0.0   0.0    2.0   0.0  0.0
1          Executing  33.0  0.0  3.0  0.0  15.0   24.0   0.0  0.0
2           Released  38.0  1.0  0.0  2.0  11.0  142.0  15.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10  102   12   16    19     20    21   22
0  Completed  12.0  0.0  0.0  0.0   0.0    2.0   0.0  0.0
1  Executing  33.0  0.0  3.0  0.0  15.0   24.0   0.0  0.0
2   Released  38.0  1.0  0.0  2.0  11.0  142.0  15.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()